# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [16]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import cv2
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [17]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [18]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 20 #experiment with the batch size
x = 30 #no of images you use for each video, or the number of frames
y = 120 # Length of image
z = 120 # Height of image
resize_dim = (120,120) #This is the dimension for resize

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [19]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(0,x)]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #Resizing the image
                    if (image.shape[0]==360):
                        image_cropped = image[50:310,50:310]
                    else:
                        image_cropped = image
                        
                    image_resized = cv2.resize(image_cropped,resize_dim, interpolation = cv2.INTER_AREA)
                    #We will normalize each of the components RGB of the image
                    #Saving the Red component separately
                    image_red = image_resized
                    image_red[:,:,1:2] = 0
                    #Saving the green separately
                    image_gr = image_resized
                    image_gr[:,:,0:2] = 0
                    #Saving the blue separately
                    image_bl = image_resized
                    image_bl[:,:,0:2] = 0
                                               
                    batch_data[folder,idx,:,:,0] = (image_red[:,:,0] - image_red.mean())/image_red.std()#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_gr[:,:,1] - image_gr.mean())/image_gr.std()#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_bl[:,:,2] - image_bl.mean())/image_bl.std()#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(folder_list)!=num_batches*batch_size):
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3))
            batch_labels = np.zeros((batch_size,5))
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #Resizing the image
                    if (image.shape[0]==360):
                        image_cropped = image[50:310,50:310]
                    else:
                        image_cropped = image
                        
                    image_resized = cv2.resize(image_cropped,resize_dim, interpolation = cv2.INTER_AREA)
                    #We will normalize each of the components RGB of the image
                    #Saving the Red component separately
                    image_red = image_resized
                    image_red[:,:,1:2] = 0
                    #Saving the green separately
                    image_gr = image_resized
                    image_gr[:,:,0:2] = 0
                    #Saving the blue separately
                    image_bl = image_resized
                    image_bl[:,:,0:2] = 0
                                               
                    batch_data[folder,idx,:,:,0] = (image_red[:,:,0] - image_red.mean())/image_red.std()#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_gr[:,:,1] - image_gr.mean())/image_gr.std()#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_bl[:,:,2] - image_bl.mean())/image_bl.std()#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [20]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [21]:
train_generator = generator(train_path, train_doc, batch_size)

In [22]:
print(train_generator)

<generator object generator at 0x7f68c5448620>


In [23]:
#next(train_generator)

# Experiment No 1

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [24]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten
from keras.layers import BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
#Model Hyperparameters are here
filtersize12 = (2,2,2)
filtersize34 = (3,3,3)

#write your model here
model = Sequential()
model.add(Conv3D(8, #number of filters 
                 kernel_size=filtersize12, 
                 input_shape=(x, y, z, 3),
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.05))
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, #Number of filters, 
                 kernel_size=filtersize12, 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.05))
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, #Number of filters 
                 kernel_size=filtersize34, 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.05))
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, #Number pf filters 
                 kernel_size=filtersize34, 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.05))
model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(500, activation='relu'))
#model.add(Dropout(0.2))

#softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [25]:
optimiser = optimizers.Adam(lr=0.001)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 30, 120, 120, 8)   200       
_________________________________________________________________
batch_normalization_4 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_4 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 15, 60, 60, 16)    1040      
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_5 (Activation)    (None, 15, 60, 60, 16)   

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [26]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [27]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [28]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 20
Epoch 1/30
 4/34 [==>...........................] - ETA: 32s - loss: 12.1025 - categorical_accuracy: 0.2000